In [52]:
from hera.shared import global_config
from hera.auth import ArgoCLITokenGenerator

global_config.host = "https://localhost:2746"
global_config.token = ArgoCLITokenGenerator
global_config.verify_ssl = False

## WorkflowTemplate <-> Pipeline & Workflow <-> Flow

WorkFlowTemplates crucially allow the definition of parameters as input variables to the entire workflow *without specifying a value*. You can also specify the default value, but it is possible to omit in cases where a sensible value just doesnt apply.

Submitting these WorkFlow templates doesnt execute anything on the ArgoWorkflow engine, but rather adds a template that future `Workflow`s can be derived from, together with configurable workflow input parameters.

This allows for the following entity mapping between bettmensch.ai pipelines and ArgoWorkflow resources:

| bettmensch.ai | ArgoWorkflow      |
| --------------|-------------------|
| `Flow`        | `WorkflowTemplate`|
| `Run`         | `Workflow`        |

## Define script templates

In [53]:
from hera.workflows import script, Parameter, models as m

@script(outputs=[Parameter(name="sum",value_from=m.ValueFrom(path="./sum.txt"))])
def add(a: float, b: float):
    with open('./sum.txt','w') as output:
        output.write(str(a + b))
        
@script(outputs=[Parameter(name="difference",value_from=m.ValueFrom(path="./difference.txt"))])
def subtract(a: float, b: float):
    with open('./difference.txt','w') as output:
        output.write(str(a - b))
        
@script(outputs=[Parameter(name="product",value_from=m.ValueFrom(path="./product.txt"))])
def multiply(a: float, b: float):
    with open('./product.txt','w') as output:
        output.write(str(a * b))

type(add), type(subtract), type(multiply)

(function, function, function)

## Define Workflow and DAG

In [54]:
from hera.workflows import WorkflowTemplate, DAG, Parameter, S3Artifact

dag_name = "2ab-minus-bcd"

with WorkflowTemplate(
    generate_name="2ab-minus-bcd-", 
    entrypoint=dag_name,
    namespace="argo",
    arguments=[Parameter(name="a"),Parameter(name="b"),Parameter(name="c"),Parameter(name="d")],
    # the Workflow referencing this template inherits the `spec` level `workflow_metadata` field from this WorkflowTemplate, so
    # when submitting via .create(), the workflow metadata does get merged into the Workflow CRD manifest before being applied to the K8s cluster
    workflow_metadata={
        'annotations':{'annotation_key_a':'annotation_value_a'},
        'labels':{'label_key_a':'label_value_a'}
    },
    # the Workflow referencing this template inherits the `spec` level `pod_metadata` field from this WorkflowTemplate, so
    # when submitting via .create(), the workflow metadata does get merged into the Pod manifest(s) before being applied to the K8s cluster.
    # for whatever reason, the (apparently?) pod level manifests shown on the ArgoWorkflow server dashboard do not show this
    pod_metadata={ 
        'annotations':{'annotation_key_b':'annotation_value_b'},
        'labels':{'label_key_b':'label_value_b'}
    }) as wt:
    
    wt_a, wt_b, wt_c, wt_d = wt.get_parameter('a').value,wt.get_parameter('b').value,wt.get_parameter('c').value,wt.get_parameter('d').value 
    
    with DAG(name=dag_name) as dag:
        double_a = multiply(
            name="multiply-2-and-a",arguments=[
                Parameter(name="a",value=2), 
                Parameter(name="b",value=wt_a)
            ]
        )
        bc = multiply(
            name="multiply-b-and-c",arguments=[
                Parameter(name="a",value=wt_b),
                Parameter(name="b",value=wt_c)
            ]
        )
        double_ab = multiply(
            name="multiply-2a-and-b",arguments=[
                Parameter(name="a",value=double_a.get_parameter('product').value),
                Parameter(name="b",value=wt_b)
            ]
        )
        bcd = multiply(
            name="multiply-bc-and-d",arguments=[
                Parameter(name="a",value=bc.get_parameter('product').value),
                Parameter(name="b",value=wt_d)
            ]
        )
        double_ab_minus_bcd = subtract(
            name="subtract-2ab-and-bcd",arguments=[
                Parameter(name="a",value=double_ab.get_parameter('product').value),
                Parameter(name="b",value=bcd.get_parameter('product').value)
            ],    
        )
        
        print(wt_b)
        print(bc.get_parameter('product').value)
        
        double_a >> double_ab
        bc >> bcd
        [double_ab, bcd] >> double_ab_minus_bcd       

{{workflow.parameters.b}}
{{tasks.multiply-b-and-c.outputs.parameters.product}}


In [55]:
dag.tasks

[Task(with_items=None, with_param=None, arguments=[Parameter(default=None, description=None, enum=None, global_name=None, name='a', value='2', value_from=None, output=False), Parameter(default=None, description=None, enum=None, global_name=None, name='b', value='{{workflow.parameters.a}}', value_from=None, output=False)], name='multiply-2-and-a', continue_on=None, hooks=None, on_exit=None, template=Script(volumes=None, volume_devices=None, volume_mounts=None, resources=None, metrics=None, active_deadline_seconds=None, affinity=None, archive_location=None, automount_service_account_token=None, daemon=None, executor=None, fail_fast=None, host_aliases=None, init_containers=None, memoize=None, annotations=None, labels=None, name='multiply', node_selector=None, parallelism=None, http=None, plugin=None, pod_spec_patch=None, priority=None, priority_class_name=None, retry_strategy=None, scheduler_name=None, pod_security_context=None, service_account_name=None, sidecars=None, synchronization=No

In [56]:
wt.templates

[DAG(metrics=None, active_deadline_seconds=None, affinity=None, archive_location=None, automount_service_account_token=None, daemon=None, executor=None, fail_fast=None, host_aliases=None, init_containers=None, memoize=None, annotations=None, labels=None, name='2ab-minus-bcd', node_selector=None, parallelism=None, http=None, plugin=None, pod_spec_patch=None, priority=None, priority_class_name=None, retry_strategy=None, scheduler_name=None, pod_security_context=None, service_account_name=None, sidecars=None, synchronization=None, timeout=None, tolerations=None, inputs=None, outputs=None, arguments=None, target=None, tasks=[Task(with_items=None, with_param=None, arguments=[Parameter(default=None, description=None, enum=None, global_name=None, name='a', value='2', value_from=None, output=False), Parameter(default=None, description=None, enum=None, global_name=None, name='b', value='{{workflow.parameters.a}}', value_from=None, output=False)], name='multiply-2-and-a', continue_on=None, hooks

In [57]:
multiply.__dict__

{'__wrapped__': <function __main__.multiply(a: float, b: float)>,
 'wrapped_function': <function __main__.multiply(a: float, b: float)>}

In [41]:
from typing import Tuple

def test_func(a: int, b: int = 1) -> Tuple[int,int]:
    return a,b

In [61]:
from hera.workflows._unparse import roundtrip
import textwrap
import inspect
test_content = roundtrip(textwrap.dedent(inspect.getsource(test_func))).splitlines()

print(f"Function content: {test_content}")

for i, line in enumerate(test_content):
    print(f"Line {i}: {line}")

Function content: ['def test_func(a: int, b: int=1) -> Tuple[int, int]:', '    return (a, b)']
Line 0: def test_func(a: int, b: int=1) -> Tuple[int, int]:
Line 1:     return (a, b)


In [62]:
test_params = inspect.signature(test_func).parameters
test_params

mappingproxy({'a': <Parameter "a: int">, 'b': <Parameter "b: int = 1">})

In [63]:
test_params['a'].default, test_params['b'].default

(inspect._empty, 1)

In [64]:
inspect.getfullargspec(test_func)

FullArgSpec(args=['a', 'b'], varargs=None, varkw=None, defaults=(1,), kwonlyargs=[], kwonlydefaults=None, annotations={'return': typing.Tuple[int, int], 'a': <class 'int'>, 'b': <class 'int'>})

In [ ]:
wt.to_file